In [1]:
import os
import time
import pandas as pd
import numpy as np
import warnings
import statsmodels.api as sm
from tqdm import tqdm
import seaborn as sns
import datetime
import time
import matplotlib.pyplot as plt
import statsmodels.stats as stat
import sys
import scipy
#import pickle5 as pickle
tqdm.pandas(desc="my bars:")
warnings.filterwarnings("ignore")

In [5]:
work_path = os.path.dirname(os.getcwd())
work_path = os.path.dirname(work_path)

In [11]:
work_path

'C:\\Users\\lenovo\\git_intro\\SabrSummerProject2022'

In [7]:
etf = pd.read_pickle(work_path+'\\03_data\\300etf.pkl')
option2207 = pd.read_pickle(work_path + '\\03_data\\300etf_option2207.pkl')

In [9]:
etf = etf.rename(columns = {'datetime':'time'})
etf['time'] = pd.to_datetime(etf['time'])
option2207['strike price'] = option2207['option_code'].apply(lambda x: int(x[-4:]))/1000
option2207['type'] = option2207['type'].apply(lambda x: 'C' if x == 1 else 'P')
option2207['time_to_mature'] = option2207['time'].apply(lambda x: x.date())
trading_date = np.sort(np.unique(option2207['time_to_mature']))
trading_interval = np.unique(option2207['time'].apply(lambda x: str(x.hour).zfill(2)+str(x.minute).zfill(2)))
option2207['time_to_mature'] = option2207['time_to_mature'].apply(lambda x: len(trading_date) + 1 - np.where(trading_date == x)[0][0])
option2207['time_to_mature'] = option2207['time_to_mature'] * len(trading_interval) + option2207['time'].apply(lambda x: len(trading_interval) - 1 - np.where(trading_interval == (str(x.hour).zfill(2)+str(x.minute).zfill(2)))[0][0])
trading_year = len(trading_interval) * 250
option2207['time_to_mature'] = option2207['time_to_mature'] / trading_year

In [10]:
option2207

,time,code,open,high,low,close,volume,option_code,type,maturity_date,strike,strike price,time_to_mature
0,2022-05-26 09:35:00,10004299.SH,0.3987,0.3987,0.3987,0.3987,1.0,510300C2207M03600,C,2022-07-27 15:30:00,3.6,3.6,0.183917
1,2022-05-26 09:35:00,10004300.SH,0.3146,0.3146,0.3025,0.3025,3.0,510300C2207M03700,C,2022-07-27 15:30:00,3.7,3.7,0.183917
2,2022-05-26 09:35:00,10004301.SH,0.2454,0.2454,0.2317,0.2361,16.0,510300C2207M03800,C,2022-07-27 15:30:00,3.8,3.8,0.183917
3,2022-05-26 09:35:00,10004302.SH,0.1808,0.1808,0.1645,0.1718,293.0,510300C2207M03900,C,2022-07-27 15:30:00,3.9,3.9,0.183917
4,2022-05-26 09:35:00,10004303.SH,0.1227,0.1244,0.1158,0.1180,220.0,510300C2207M04000,C,2022-07-27 15:30:00,4.0,4.0,0.183917
...,...,...,...,...,...,...,...,...,...,...,...,...,...
52699,2022-07-27 15:00:00,10004334.SH,0.4293,0.4293,0.4293,0.4293,0.0,510300P2207M04700,P,2022-07-27 15:30:00,4.7,4.7,0.008000
52700,2022-07-27 15:00:00,10004379.SH,0.0001,0.0001,0.0001,0.0001,9.0,510300C2207M04800,C,2022-07-27 15:30:00,4.8,4.8,0.008000
52701,2022-07-27 15:00:00,10004380.SH,0.5287,0.5287,0.5287,0.5287,0.0,510300P2207M04800,P,2022-07-27 15:30:00,4.8,4.8,0.008000
52702,2022-07-27 15:00:00,10004391.SH,0.0001,0.0001,0.0001,0.0001,9.0,510300C2207M04900,C,2022-07-27 15:30:00,4.9,4.9,0.008000


In [12]:
#####option_model######
sys.path.append(work_path + '\\02_src\\OptionModel')

In [24]:
from sabr_bywynn import BlackScholes, SABR

In [18]:
option2207['date'] = option2207['time'].apply(lambda x: x.date())
option2207 = pd.merge(option2207, etf.rename(columns = {'close':'spot price'})[['time','spot price']], how = 'left', on  = 'time')

In [25]:
def pandas_apply_sabr(temp):
    global params
    global r
    
    # drop deep itm call
    deepcall = temp.loc[(temp['type'] == 'C')&(temp['strike price'] < temp['spot price']-0.3), :]
    dropindex = deepcall.loc[deepcall['close']+deepcall['strike price'] > deepcall['spot price'], :].index
    temp = temp.drop(dropindex)
    # calculate black IV
    y = temp['strike price'].values
    expiry = temp['time_to_mature'].values[0]
    spot_price = temp['spot price'].values[0]
    isCall = temp['type'].apply(lambda x: True if x == 'C' else False)
    price = temp['close'].values
    BS = BlackScholes(y, expiry, spot_price, 0.04, isCall)
    bvol = BS.blackIV(price)
    # drop vol outlier
    bvol[bvol<0.08] = bvol[bvol>0.08].min()
    bvol[bvol>0.5] = np.NAN
    # check
    if len(bvol[~np.isnan(bvol)]) < 3:
        return temp
    # drop 10 delta
    bdelta = BS.blackDelta(bvol)
    argfit = (np.abs(bdelta) > 0.1)&(np.abs(bdelta) < 0.9)
    # fit sabr
    y = y[argfit]
    isCall = isCall[argfit]
    price = price[argfit]
    # volume = temp['volume'].values[argfit]
    # volume[volume < 1] = 1
    atm_spread = y[np.abs(y - spot_price).argmin()] - spot_price
    atm_index_c = np.where(y[isCall] == spot_price + atm_spread)
    atm_index_c = np.where(price == price[isCall][atm_index_c])
    atm_index_p = np.where(y[~isCall] == spot_price + atm_spread)
    atm_index_p = np.where(price == price[~isCall][atm_index_p])
    if atm_spread < 0:
        atm_index = atm_index_p
    else:
        atm_index = atm_index_c
    # use atm put call parity to calculate riskfree interest rate
    r_plus = -np.log((price[atm_index_p] - price[atm_index_c] + spot_price)/y[atm_index]) / expiry
    r = (4*r+r_plus[0])/5
    sabr = SABR(y, expiry, spot_price, r, isCall)
    param= sabr.haganFit_delta(sabr.BS, price, atm_index)
    temp = temp.loc[argfit, :]
    params[temp['time'].values[0]] = param
    temp['black vol'] = bvol[argfit]
    temp['sabr vol'] = sabr.haganLogNormalApprox(sabr.BS, param[0], param[1], param[2], param[3])
    temp['delta'] = sabr.computeSABRdelta(sabr.BS, param[0], param[1], param[2], param[3])
    temp['vega'] = sabr.computeSABRvega(sabr.BS, param[0], param[1], param[2], param[3])
    temp['theta'] = sabr.computeSABRtheta(sabr.BS, param[0], param[1], param[2], param[3])
    temp['gamma'] = sabr.computeSABRgamma(sabr.BS, param[0], param[1], param[2], param[3])
    temp['riskfree rate'] = r
    
    return temp

In [20]:
date0 = np.unique(option2207['date'])[-7]
date1 = np.unique(option2207['date'])[-1]
insample = option2207.loc[(option2207['date'] > date0)&(option2207['date'] <= date1), :]

In [30]:
params = {}
r = -0.01
insample_sabr = insample.groupby('time').progress_apply(pandas_apply_sabr).reset_index(drop = True)

my bars::   1%|▌                                                                       | 2/288 [00:07<17:38,  3.70s/it]

Optimization terminated successfully


my bars::   1%|▊                                                                       | 3/288 [00:14<25:07,  5.29s/it]

Optimization terminated successfully


my bars::   1%|█                                                                       | 4/288 [00:22<29:01,  6.13s/it]

...

Optimization terminated successfully


my bars:: 100%|██████████████████████████████████████████████████████████████████████| 288/288 [43:37<00:00,  7.76s/it]

Optimization terminated successfully


my bars:: 100%|██████████████████████████████████████████████████████████████████████| 288/288 [43:45<00:00,  9.12s/it]

Optimization terminated successfully


In [34]:
insample_sabr.to_pickle(work_path+"\\03_data\\sample_sabr.pkl")

In [35]:
option2207_sabr = option2207.groupby('time').progress_apply(pandas_apply_sabr).reset_index(drop = True)

my bars::   0%|▏                                                                      | 7/2112 [00:07<29:20,  1.20it/s]

Optimization terminated successfully
Optimization terminated successfully
Optimization terminated successfully
Optimization terminated successfully
Optimization terminated successfully
Optimization terminated successfully
Optimization terminated successfully
Optimization terminated successfully
Optimization terminated successfully


my bars::   1%|▎                                                                     | 11/2112 [00:07<15:53,  2.20it/s]

Optimization terminated successfully
Optimization terminated successfully
Optimization terminated successfully


my bars::   1%|▋                                                                     | 19/2112 [00:22<38:57,  1.12s/it]

...

my bars::  11%|███████▌                                                           | 240/2112 [02:52<2:48:36,  5.40s/it]

Optimization terminated successfully
Optimization terminated successfully


my bars::  11%|███████▉                                                             | 242/2112 [03:08<24:14,  1.29it/s]

Optimization terminated successfully


KeyError: 'time'